<a href="https://colab.research.google.com/github/lingpri/congenial-sniffle/blob/main/Priya_Feb2_U1_MH2_AptitudeClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

## Problem Statement

The problem is to identify the subcategory and classify the question based on the group it belongs to.



## Learning Objectives

At the end of the experiment, you will be able to understand:

*   Beautiful Soup
*   Use NLTK package
*   Text Representation
*   Classification

In [ ]:
#@title  Mini Hackathon Walkthrough
from IPython.display import HTML

HTML("""<video width="320" height="240" controls>
  <source src="https://cdn.talentsprint.com/talentsprint/archives/sc/aiml/aiml_batch_15/preview_videos/Mini_Hackathon_Aptitude_Classification.mp4" type="video/mp4">
</video>
""")

## Dataset
Being able to classify the questions will be difficult in natural language processing. The dataset is taken from the TalentSprint aptitude questions which contains more than 20K questions.

## Description
This dataset has the following columns:
1. **Category:** Gives the high-level categorization of the question
2. **Sub-Category:** Determines the type of questions
3. **Article:** Gives the article name of the question
4. **Questions:** Questions are listed
5. **Answers:** Contains answers


### Grading = 20 Marks

In [ ]:
#@title Download the datasets
from IPython import get_ipython

ipython = get_ipython()

def setup(): 
    ipython.magic("sx wget https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/Aptitude_Classification_data.csv")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/Mentors_Test_Data.csv")
    from IPython.display import HTML, display
    print("Setup completed successfully")
    return

setup()

Setup completed successfully


In [ ]:
# Import Python Libraries
from bs4 import BeautifulSoup
import nltk
import re
import string
import warnings
import numpy as np
import pandas as pd
from collections import Counter
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

##   **Stage 1**:  Dataset Preparation

### 1 Mark -> Load the data set and prepare the data based on group allocation. 

Each group should consider their respective sub-categories as mentioned below:

> Team A = Groups 1, 4, 7, 10, 13, 16, 19, 22;   &nbsp; &nbsp;   Sub-Category = Misspell words, Algebra, Percentages, Mathematical operations, Probability

> Team B = Groups 2, 5, 8, 11, 14, 17, 20, 23; &nbsp; &nbsp;   Sub-Category = Finding Errors, Ratio and Proportion, Logarithms, Time and Distance, Simple and Compound Interest

> Team C = Groups 3, 6, 9, 12, 15, 18, 21, 24;  &nbsp; &nbsp;  Sub-Category =  Synonyms and Antonyms, Time and Work, Permutations and Combinations, LCM and HCF, Profit and Loss


**Hint:** &nbsp; To access Sub-Categories from given Data, refer [link](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html)

In [ ]:
# YOUR CODE HERE TO LOAD THE APTITUDE CLASSIFICATION DATASET & EXTRACT THE DATA BASED ON YOUR SUB-CATEGORIES
import pandas as pd
import numpy as np

team_10_sub_cat = ['']

data = pd.read_csv("Aptitude_Classification_data.csv")
sub_cat = np.unique(data["Sub-Category"])
team_10_sub_cat = ["Misspell words","Algebra","Percentages","Mathematical operations","Probability"]
np.unique(sub_cat)
data = data[data["Sub-Category"].isin(team_10_sub_cat)]
data.shape , sub_cat, team_10_sub_cat # 1520 rows and 5 columns
data.head()




,Category,Sub-Category,Article,Questions,Answers
0,Verbal,Misspell words,chapter 1,Which of the following is correct?\n\n\n\n\n,2
10,Reasoning,Mathematical operations,Mathematical Operations,"If A means +, B means -, C means X, D means /...",2
11,Quantitative,Probability,AXPTP04,A class consists of 10 boys and 8 girls. Three...,2
16,Verbal,Misspell words,chapter 1,Which of the following is correct?\n\n\n\n\n,2
19,Quantitative,Probability,AXPTP04,Find the probability that when a hand of 7 car...,2


## **Stage 2:** Data Pre-Processing

### 3 Marks -> Clean and Transform the data into a specified format

*   Remove the rows of the Questions column which contains blank / NaN.


*   Few set of questions have HTML tags within the question.
  - You can use Beautiful Soup library to convert HTML into text (Refer **"Dealing with HTML"** section from this [link](https://www.nltk.org/book/ch03.html).)


*  Consider Question column as feature and Sub-category as target variable. Convert Sub-category into numerical.

*  Drop the unwanted columns


  **Hint:** Use Label Encoder for obtaining a numeric representation, refer to the [link](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html). 

In [ ]:
# YOUR CODE HERE for BeatifulSoup
# https://datascience.stackexchange.com/questions/37878/difference-between-isna-and-isnull-in-pandas
data["Questions"].isnull().sum(axis=0) # 5 rows were found with blank/nan
data = data.dropna()
data["Questions"].isnull().sum(axis=0)
data.shape # 5 rows were dropped hence 1515 

features = data[["Questions"]]
target = data[["Sub-Category"]]

from bs4 import BeautifulSoup as bs

features['Questions'] = [BeautifulSoup(text).get_text() for text in features['Questions'] ]
features.head()










,Questions
0,Which of the following is correct?\n\n\n\n\n
10,"If A means +, B means -, C means X, D means /..."
11,A class consists of 10 boys and 8 girls. Three...
16,Which of the following is correct?\n\n\n\n\n
19,Find the probability that when a hand of 7 car...


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder() # DO NOT CHANGE THIS LINE as we will be using for the Test evaluation.

# YOUR CODE HERE for Fit label encoder and return encoded labels

data['Sub-Category_Cat'] = le.fit_transform(target['Sub-Category'])

data.head()



,Category,Sub-Category,Article,Questions,Answers,Sub-Category_Cat
0,Verbal,Misspell words,chapter 1,Which of the following is correct?\n\n\n\n\n,2,2
10,Reasoning,Mathematical operations,Mathematical Operations,"If A means +, B means -, C means X, D means /...",2,1
11,Quantitative,Probability,AXPTP04,A class consists of 10 boys and 8 girls. Three...,2,4
16,Verbal,Misspell words,chapter 1,Which of the following is correct?\n\n\n\n\n,2,2
19,Quantitative,Probability,AXPTP04,Find the probability that when a hand of 7 car...,2,4


## **Stage 3:** Text representation using Bag of Words (BOW)

### 3 Marks -> a) Get valid words from all questions & add them to a list.


Treat each question as a separate document and get the list of words using the following:
1.   Split the sentence into words

2.   Remove Stop words. Use NLTK packages for getting the Stop words.

3.   Replace proper names with "name" 
  - Example: "Rahul" -> "name"
       
4.   Remove the single white space character (\n, \t, \f, \r), refer [link](https://developers.google.com/edu/python/regular-expressions)

5.   Ignore words whose length is less than 3 (Eg: 'is', 'a').

6.   Remove punctuation and non-alphabetic words

7.   Convert the text to lowercase

8.   Use the Porter Stemmer to normalize the words


Refer [link](https://www.nltk.org/book/ch03.html) for extracting the words.

Refer [link](https://medium.com/free-code-camp/an-introduction-to-bag-of-words-and-how-to-code-it-in-python-for-nlp-282e87a9da04) for more information.

In [ ]:
remove_punctuation = str.maketrans('', '', string.punctuation)
remove_digits = str.maketrans('', '', string.digits) 

In [ ]:
import re
from nltk.corpus import stopwords
from nltk import PorterStemmer
from nltk.tag import pos_tag

porter = nltk.PorterStemmer()



def extract_words(question):
    # YOUR CODE HERE
    # Hint: Extract words for each question using the above 8 instructions.
    words  = re.sub("[^\w]"," ",question.strip()).split()

    ignore = set(stopwords.words('english'))
    words  = [w for w in words if w not in ignore]

    parts_of_speech = pos_tag(words)
    
    words  = [word if pos != 'NNP' else 'name' for word,pos in parts_of_speech ] 

    words  = [w.translate(remove_punctuation) for w in words if w not in string.punctuation ]

    words  = [w.translate(remove_digits) for w in words if w not in string.digits ]
            
    words  = [word for word in words if len(word) > 2]
    
    words  = [word.lower() for word in words]

    words  = [porter.stem(i) for i in words ]
    return set(words)

In [ ]:
def tokenize(allquestions):
  valid_words = []
  for question in allquestions:
    words = extract_words(question)
    valid_words.extend(words)
  return set(valid_words)

In [ ]:
# Use the function to extract words for all questions
# YOUR CODE HERE
questions = np.asarray(features)
questions = list(np.concatenate(questions))

vocab = tokenize(questions)
len(vocab)







1248

**bold text**### 4 Marks -> b) Generate vectors that can be used by the machine learning algorithm


1.   The length of the vector for each question will be the length of the valid words. Initialize each vector with all Zeros

2.   Compare each valid word with the words in question and generate the vectors based on the counter frequency of the word in that question.



In [ ]:
def generate_vectors(question):
    # YOUR CODE HERE
    # Hint: Initialize each vector with all zeros.
    bag_vector = np.zeros(len(vocab))

    # Extracting words for each question and count the words
    words = extract_words(question)
    word_dict = Counter(words)
    

    for i,word in enumerate(vocab):
        for w in words:
              if word == w:              
                  bag_vector[i] = word_dict[word]   
    # YOUR CODE HERE 
    # Hint: If the word is in valid words then generate the vectors based on the counter frequency of the word in that question.
    return np.array(bag_vector)
    

In [ ]:
# Use the above function for collecting the vectors of all questions into a list.
# YOUR CODE HERE
vector_list = []
for question in questions:
    vector_list.append(generate_vectors(question))

len(vector_list),len(data)
data['Vector'] = vector_list

In [ ]:
import pandas as pd

type(vector_list[0]), len(vocab)

df = pd.DataFrame(vector_list) 
df.columns = vocab

df.head(50)



,half,light,mix,reach,taxabl,home,pint,denomin,basic,honour,age,rectangular,championship,whole,independ,accord,run,machin,unchang,consecut,build,averag,need,graduat,out,togeth,board,addit,contest,morn,dime,effect,own,intern,annual,coupon,incur,send,colleg,hors,...,collect,correct,rectangl,wage,back,current,trial,obtain,queen,all,violet,label,diamond,when,give,activ,odd,milk,repli,consid,paint,follow,valid,bicycl,spelt,estat,societi,institut,unfair,maximum,particip,notebook,shall,thi,singl,grown,gold,mani,dad,choos
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(questions).toarray()


X.shape



(1515, 2332)

## **Stage 4:** Classification

### 3 Marks -> Perform a Classification 

1.   Identify the features and labels

2.   Use train_test_split for splitting the train and test data

3.   Fit your model on the train set using fit() and perform prediction on the test set using predict()

4. Get the accuracy of the model

## Expected Accuracy above 90%


In [ ]:
from sklearn.model_selection import train_test_split


y = data["Sub-Category_Cat"]

#print(sum(X.iloc[0]))

X_train,X_test,y_train,y_test = train_test_split(df,y,test_size=0.2, random_state=40)

X_train.shape, X_test.shape

k = 3

neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train,y_train)
y_pred = neigh.predict(X_test)
print(accuracy_score(y_test,y_pred))

clf = DecisionTreeClassifier(random_state=100)
clf = clf.fit(X_train,y_train)
y_pred1 = clf.predict(X_test)
print(accuracy_score(y_test,y_pred1))






0.7623762376237624
0.9141914191419142


## **Stage 5:** Evaluation (This is for Mentors)

### 6 Marks -> Evaluate with the given test data 

1.  Loading the Test data

2.  Converting the Test data into vectors

3.  Pass through the model and verify the accuracy

## Expected Accuracy above 90%


In [ ]:
# YOUR CODE HERE for selecting the trained classifier model, eg: MODEL = decision_tree
MODEL = clf

Test_data = pd.read_csv("Mentors_Test_Data.csv")
Test_data = Test_data[Test_data['Sub-Category'].isin(le.classes_)]
labels = le.transform(Test_data['Sub-Category'])
Test_questions= Test_data['Questions']

Test_BOW=[]
for TQ in Test_questions: 
  Test_vectors = generate_vectors(TQ) 
  Test_BOW.append(Test_vectors)

predict = MODEL.predict(Test_BOW) 
accuracy_score(labels, predict)

0.9113924050632911